In [ ]:
from synthetic_generator import generate_synthetic_dataset
from pre_MEDS import generate_meds_preprocessed
import os

os.makedirs("input", exist_ok=True)
os.makedirs("intermediate", exist_ok=True)
os.makedirs("output", exist_ok=True)

df_input = generate_synthetic_dataset(3, output_csv="input/syn_data.csv")
df_intermediate = generate_meds_preprocessed(df_input, output_path="intermediate")


In [ ]:
display(df_intermediate[0])

In [ ]:
display(df_intermediate[2])

In [ ]:
# from MEDS_transforms.runner import main

# main([
#     "pkg://MEDS_extract.configs._extract.yaml",
#     "--overrides",
#     "input_dir=intermediate",
#     "output_dir=output",
#     "event_conversion_config_fp=MESSY.yaml",
#     "dataset.name=Neurovasc",
#     "dataset.version=1.0",
# ])


In [ ]:
import polars as pl

train_data = pl.read_parquet(str("output/data/train/0.parquet"))
train_data

In [ ]:
pl.read_parquet(str("intermediate/patients.parquet"))

In [ ]:
pl.read_parquet(str("output/metadata/codes.parquet"))

In [ ]:
from meds2rdf import MedsRDFConverter

# Initialize the converter with the path to your MEDS dataset directory
converter = MedsRDFConverter("output")

# Convert the dataset into an RDF graph
graph = converter.convert(
    include_splits=True,
)

# Serialize the graph to different formats
graph.serialize(destination="output/rdf/output_dataset.ttl", format="turtle")
graph.serialize(destination="output/rdf/output_dataset.xml", format="xml")
graph.serialize(destination="output/rdf/output_dataset.nt", format="nt")

print("Conversion complete! RDF files saved.")

In [ ]:
from pyshacl import validate

data_graph = "output/rdf/output_dataset.ttl"
shacl_graph = "https://raw.githubusercontent.com/albertomarfoglia/meds-ontology/refs/heads/main/shacl/meds-shapes.ttl"

output_graph = validate(data_graph, shacl_graph=shacl_graph, advanced=True)

print(output_graph[2])